# 0. Initialization
## 0.1. Imports

In [1]:
import scanpy as sc
import scrublet as scr
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import gseapy as gp
from scipy.sparse import csc_matrix
import anndata

## 0.2. Load file

In [2]:
# adata = anndata.read_h5ad("../assets/BRCA_GSE161529_expression_skc.h5ad")
adata = anndata.read_h5ad("../assets/BRCA_GSE161529_expression_skc_doublet.h5ad")

In [3]:
adata

AnnData object with n_obs × n_vars = 332168 × 27188
    obs: 'doublet_score', 'doublet_ghost_15'
    var: 'gene_ids', 'gene_symbols', 'feature_types', 'genome'

# 1. Process file

In [4]:
## -- convert sparse array to dense
#X_dense = adata.X.toarray()

In [5]:
num_cells = adata.shape[0]

## 1.0. Check for missing values

In [6]:
# missing_percentage = np.sum(np.isnan(X_dense), axis=0) / num_cells * 100

## 1.1. Check for zero counts

In [7]:
# zero_percentage = np.sum(X_dense == 0, axis=0) / num_cells * 100

## 1.2. Check for negative values

In [8]:
# negative_percentage = np.sum(X_dense < 0, axis=0) / num_cells * 100

## 1.3. Write checks to disk

In [9]:
# np.save('../assets/missing_percentage.npy', missing_percentage)

In [10]:
# np.save('../assets/zero_percentage.npy', zero_percentage)

In [11]:
# np.save('../assets/negative_percentage.npy',negative_percentage)

## 1.4. Read from disk and analyze

In [12]:
missing_percentage = np.load('../assets/missing_percentage.npy')

In [13]:
zero_percentage = np.load('../assets/zero_percentage.npy')

In [14]:
negative_percentage = np.load('../assets/negative_percentage.npy')

In [15]:
np.sum(missing_percentage), np.sum(zero_percentage), np.sum(negative_percentage)

(np.float64(0.0), np.float64(2544075.847161677), np.float64(0.0))

In [16]:
np.sum(zero_percentage > 0), np.sum(zero_percentage<=0)

(np.int64(27188), np.int64(0))

## 1.4. Log normalize gene expression

In [17]:
# type(X_dense)

In [18]:
# sc.pp.normalize_total(adata, target_sum=1e4)
# sc.pp.log1p(adata)

In [19]:
# # STEP 2: Calculate mitochondrial content
# adata.var['mt'] = adata.var_names.str.startswith('MT-')
# sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
# # Store mitochondrial percentage
# adata.obs['pct_mito'] = adata.obs['pct_counts_mt']

# 2. Feature Engineering - Independent Variables
Note that we do *not* normalize the counts for engineering our independent variables.

Doublet score is calculated using a separate python script because of memory limitations with jupyter.

## 2.0. Mitochondrial score 

In [20]:
# Calcualte the mitochondrial content
adata.var['mt'] = adata.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [25]:
adata.obs['mito_ghost_15'] = (adata.obs['pct_counts_mt'] > 15.0).astype(int)

## 2.1. Ambient score

In [26]:
adata.obs['ambient_ghost'] = (adata.obs['doublet_ghost_15'] + adata.obs['mito_ghost_15']) / 2

In [22]:
# adata.obs['doublet_score'] = doublet_scores
# adata.obs['doublet_ghost'] = predicted_doublets.astype(int)

# 3. Feature Engineering - Dependent Variables
## 3.0. Normalize gene expression data

In [44]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

## 3.1. Get hallmark genesets

In [49]:
genesets_of_interest = {
    'HALLMARK_G2M_CHECKPOINT':'G2-M Checkpoint',
    'HALLMARK_E2F_TARGETS': 'E2F Targets',
    'HALLMARK_DNA_REPAIR': 'DNA Repair',
    'HALLMARK_UNFOLDED_PROTEIN_RESPONSE': 'Unfolded Protein Response',
}


In [50]:
hallmark_gene_sets = gp.get_library(name="MSigDB_Hallmark_2020", organism="Human")

## 3.2. cd274_expr

In [45]:
'CD274' in adata.var_names

True

In [46]:
adata.obs['cd274_expr'] = adata[:, 'CD274'].X.toarray().flatten()

## 3.3. hprt_expr

In [48]:
'HPRT' in adata.var_names

False

## 3.4. g2m_score

In [51]:
g2m_gene_list = hallmark_gene_sets.get(genesets_of_interest['HALLMARK_G2M_CHECKPOINT'])

In [53]:
sc.tl.score_genes(adata, gene_list=g2m_gene_list, score_name='g2m_score')

## 3.5. e2f_score

In [54]:
e2f_gene_list = hallmark_gene_sets.get(genesets_of_interest['HALLMARK_E2F_TARGETS'])
sc.tl.score_genes(adata, gene_list=e2f_gene_list, score_name='e2f_score')

## 3.6. dr_score

In [55]:
dr_gene_list = hallmark_gene_sets.get(genesets_of_interest['HALLMARK_DNA_REPAIR'])
sc.tl.score_genes(adata, gene_list=dr_gene_list, score_name='dr_score')

## 3.7. upr_score

In [56]:
upr_gene_list = hallmark_gene_sets.get(genesets_of_interest['HALLMARK_UNFOLDED_PROTEIN_RESPONSE'])
sc.tl.score_genes(adata, gene_list=upr_gene_list, score_name='upr_score')

## 3.8. rpl13a_expr

In [58]:
'RPL13A' in adata.var_names

True

In [59]:
adata.obs['RPL13A'] = adata[:, 'RPL13A'].X.toarray().flatten()

## 3.9. tbp_expr

In [60]:
'TBP' in adata.var_names

True

In [61]:
adata.obs['TBP'] = adata[:, 'TBP'].X.toarray().flatten()

# 4. Write out with features

In [62]:
adata.write("../assets/BRCA_GSE161529_expression_skc_engineered.h5ad")